In [ ]:
import pandas as pd
import numpy as np
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm import tqdm 
import time
import random
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [ ]:
def extract_company(result):
    try:
        return result.find('span',{'class':'companyName'}).text
    except:
        return None

In [ ]:
def extract_salary(result):
    try:
        return result.find('div',{'class':'attribute_snippet'}).text
    except:
        return None

In [ ]:
def extract_location(result):   
    try:
        return result.find('div',{'class':'companyLocation'}).text
    except:
        return None

In [ ]:
def extract_job(result):
    try:
        return result.find('h2', attrs={'class':'jobTitle'}).text
    except:
        return None

In [ ]:
def extract_url(result):
  try:
    atag = result.h2.a
    job_url = 'https://www.indeed.com' + atag.get('href')
    return job_url
  except:
    return None 

In [ ]:
def extract_rating(result):
    try:
        return result.find('span', attrs={'class':'ratingNumber'}).text
    except:
        return None

In [ ]:
def extract_snippet(result):
    try:
        return result.find('div', attrs={'class':'job-snippet'}).text.replace('\n', ' ')
    except:
        return None

In [ ]:
def extract_date(result):
  try:
    return result.find('span', {'class':'date' }).text.replace('\n', '')
  except:
    return None

In [ ]:
job = 'data+scientist'
url_template = "https://www.indeed.com/jobs?q="+ job + "&l&vjk=5015ead57ec8b7ad"
max_results_per_city = 1000
results = []

In [ ]:
job = 'data+scientist'
url_template = "https://www.indeed.com/jobs?q="+ job + "&l&vjk=5015ead57ec8b7ad"
max_results_per_city = 1000
results = []

for city in ['Remote']:
    for start in tqdm(range(0, max_results_per_city, 10)):
        time.sleep(random.randint(1,2))
        r1 = requests.get(url_template.format(city,start))

        soup = BeautifulSoup(r1.text, 'html.parser')

        jobs = soup.find_all(name='div', attrs={'class':"slider_container"})
        results.extend(jobs)

In [ ]:
df = pd.DataFrame({"Job Title":extract_job(result),"Company":extract_company(result),
     "Location":extract_location(result),"Salary":extract_salary(result),
      "Rating":extract_rating(result),"Job_Description":extract_snippet(result), "Urls":extract_url(result), "Date": extract_date(result)} for result in results)

In [ ]:
# df.to_csv('indeed.csv', index=False, encoding='utf-8')


In [ ]:
#df = pd.read_csv('indeed.csv')

In [ ]:
def extract_url(job_url):
    r1 = requests.get(job_url.format(start))
    soup = BeautifulSoup(r1.text, 'html.parser')
    result = soup.find_all(name='div', attrs={'class':"icl-u-lg-hide"})
    # extract_url = result.get('href')
    return result 

In [ ]:
df.info()

In [ ]:
df = df.reset_index()

In [ ]:
df.head()

In [ ]:
df['Urls'][0]

In [ ]:
external_URL = []
for n in range(int(len(df))):
    external_URL.append(str(extract_url(df['Urls'][n])))

In [ ]:
df['external_URL'] = 0    

In [ ]:
len(df)

In [ ]:
for n in range(len(external_URL)):
    df['external_URL'][n] = external_URL[n]    

In [ ]:
for n in range(len(df)):
    if df['external_URL'][n] != 0:
        before, sep, after = df['external_URL'][n].partition('"icl-Button icl-Button--primary icl-Button--lg icl-Button--block jobsearch-CallToApply-applyButton-newDesign" href="')
        if len(after) > 0:
            df['external_URL'][n] = after

In [ ]:
for n in range(len(df)):
    if df['external_URL'][n] != 0:
        before, sep, after = df['external_URL'][n].partition('" referrerpolicy="origin" rel="noopener"')
        if len(after) > 0:
            df['external_URL'][n] = before

In [ ]:
df.head(12)

In [ ]:
df = df.reset_index()

In [ ]:
df.info()

In [ ]:
for n in range(len(df)):
    if df['external_URL'][n] != 0:
        df.external_URL[n].strip()

In [ ]:
df.head()

In [ ]:
df = df=df.drop_duplicates()

In [ ]:
df.to_csv('indeed3.csv', index=False, encoding='utf-8')


In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install gspread

In [ ]:
import gspread
from google.oauth2.service_account import Credentials

In [ ]:
def create_credentials():

    scopes = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive'
        ]
        
    credentials = Credentials.from_service_account_file(
            'xxxxxxxxx.json',
            scopes=scopes
        )
    
    return gspread.authorize(credentials)

In [ ]:
def update_spreadsheet(id_sheet, id_spreadsheet, df):
    gc = create_credentials()
    
    gc = gc.open_by_key(id_spreadsheet)
    
    sheet = gc.get_worksheet(id_sheet)
    
    sheet.update(([df.columns.values.tolist()] + df.values.tolist()))


In [ ]:
update_spreadsheet(0, "xxxxxxxx", df)